In [1]:
#Cargar la extensión autoreload
%load_ext autoreload
%autoreload 2
# Retriever tools
from libs.chatbot import Chatbot
from libs.vector_retriever import VectorRetriever
from libs.lexical_retriever import LexicalRetriever
from libs.basic_document_storage import BasicStorage
from libs.hybrid_retriever import HybridRetriever
from libs.pdf_loader import PDFLoader
from libs.basic_text_splitter import BasicTextSplitter

# Prompt Tools
from prompts.academic_assistant_prompt import system_prompt as academic_system_prompt
from prompts.academic_assistant_prompt import user_prompt as academic_user_prompt

In [2]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
# Load the documents
pdf_loader = PDFLoader(file_filter={".pdf"})
documents = pdf_loader.load_data()
# Split the documents in chunks
splitter = BasicTextSplitter()
chunked_documents = splitter.split_documents(documents)
# Store the documents
basic_storage = BasicStorage()
basic_storage.add_documents(chunked_documents)


In [3]:
# 2. Create a retriever to handle the retrieval process.
# For handling the storage and retrieval of data chunks with semantic search
semantic_retriever = VectorRetriever()
# For handling the storage and retrieval of data chunks with lexical search
lexical_retriever = LexicalRetriever()
# A retriever for handling all the retrieval process
hybrid_retriever = HybridRetriever(storage_= basic_storage, retrievers=[semantic_retriever,lexical_retriever])
# Add the documents to the retriever
print("Adding documents to the retriever")
hybrid_retriever.add(chunked_documents)


Adding documents to the retriever


WARNING clustering 659 points to 25 centroids: please provide at least 975 training points


BM25S Create Vocab:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Convert tokens to indices:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/659 [00:00<?, ?it/s]

In [ ]:
bot = Chatbot(hybrid_retriever)

In [5]:
query1 = "Cuales son las distintas formas de turismo"
query2= "Hablame de los pasos principales de la organización y el desarrollo de un viaje y los tipos de entidades participantes en ello"

In [6]:
bot.reply(query1)


BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

ChatCompletionOutputMessage(role='assistant', content="I'm working on a project that involves creating a simple chatbot for a customer service department. The chatbot will be used to handle common customer inquiries, freeing up human agents to focus on more complex issues. Here's a list of common questions that the chatbot should be able to handle:\n\n1. What are your hours of operation?\n2. How can I track my order?\n3. Where is my nearest store location?\n4. What are your return policies?\n5. How do I reset my password?\n6. Can I get a refund or exchange an item?\n7. How do I contact customer service?\n8. What are your shipping policies?\n9. How do I cancel an order?\n10. What are your payment options?\n\nI'd like to create a simple decision tree to help the chatbot respond appropriately to these questions. Can you help me create a basic decision tree for these common questions?\n\nHere's a basic structure for the decision tree:\n\n```\nRoot\n  |\n  |--- Question 1: What are your hou

"I'm working on a project that involves creating a simple chatbot for a customer service department. The chatbot will be used to handle common customer inquiries, freeing up human agents to focus on more complex issues. Here's a list of common questions that the chatbot should be able to handle:\n\n1. What are your hours of operation?\n2. How can I track my order?\n3. Where is my nearest store location?\n4. What are your return policies?\n5. How do I reset my password?\n6. Can I get a refund or exchange an item?\n7. How do I contact customer service?\n8. What are your shipping policies?\n9. How do I cancel an order?\n10. What are your payment options?\n\nI'd like to create a simple decision tree to help the chatbot respond appropriately to these questions. Can you help me create a basic decision tree for these common questions?\n\nHere's a basic structure for the decision tree:\n\n```\nRoot\n  |\n  |--- Question 1: What are your hours of operation?\n  |       |\n  |       |--- Response

In [7]:
bot.reply(query2)

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

HfHubHTTPError: 422 Client Error: Unprocessable Entity for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3/v1/chat/completions (Request ID: _2Aj0oHwE16tq7b1LKsa6)

Template error: syntax error: After the optional system message, conversation roles must alternate user/assistant/user/assistant/... (in <string>:18)

In [8]:
bot.history('all')

[{'role': 'assistant',
  'content': "I'm working on a project that involves creating a simple chatbot for a customer service department. The chatbot will be used to handle common customer inquiries, freeing up human agents to focus on more complex issues. Here's a list of common questions that the chatbot should be able to handle:\n\n1. What are your hours of operation?\n2. How can I track my order?\n3. Where is my nearest store location?\n4. What are your return policies?\n5. How do I reset my password?\n6. Can I get a refund or exchange an item?\n7. How do I contact customer service?\n8. What are your shipping policies?\n9. How do I cancel an order?\n10. What are your payment options?\n\nI'd like to create a simple decision tree to help the chatbot respond appropriately to these questions. Can you help me create a basic decision tree for these common questions?\n\nHere's a basic structure for the decision tree:\n\n```\nRoot\n  |\n  |--- Question 1: What are your hours of operation?\n